In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import argparse
import logging
import glob

from utils.ingest import ingest_document
from utils.database_utils import generate_database_and_retriever, populate_database
from utils.summarize import summarize_objects


In [46]:
LOGGER = logging.getLogger(__name__)

In [47]:
folder = "./documents"
assert folder is not None and folder != "", "Folder needs to be specified"
LOGGER.info("Parsing all pdf documents in {}".format(folder))
all_documents = glob.glob(f"{folder}/*.pdf")

In [48]:
all_texts = []
all_tables = []
all_images = []

for doc in all_documents:
    LOGGER.info("Parsing document: {}".format(doc))
    text_objs, table_objs, images_objs = ingest_document(doc)
    all_texts.extend(text_objs)
    all_tables.extend(table_objs)
    all_images.extend(images_objs)

In [ ]:
all_texts, all_images, all_tables = summarize_objects(all_texts, all_images, all_tables)

100%|██████████| 2/2 [00:13<00:00,  6.59s/it]


In [70]:
data_base = "./localdb"
retriever = generate_database_and_retriever(main_folder=data_base)

InternalError: Database error: error returned from database: (code: 1) no such table: tenants

In [69]:
retriever = populate_database(retriever, all_texts, all_images, all_tables)

In [ ]:
# For each chunk you need to extract the Graph:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm = ChatOllama(temperature=0, model="gemma3:12b")
graph_transformer = LLMGraphTransformer(
    llm=llm, node_properties=["name", "type", "canonical_name"]
)
